### Imports

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast
import os, math, json, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.utils.class_weight import compute_class_weight


### --- 2. Configuration ---


In [ ]:
# Path to your landmark file (created by the first notebook)
LANDMARK_FILE = r'../data/Landmarks_GCN_augmented.npz'
# Path to the JSON file defining the 300-class dataset
SPLIT_FILE = r'../data/nslt_300.json'
# Path to a new folder where all outputs (models, logs) will be saved
OUTPUT_DIR = r'../outputs/stgcn_300/'

# --- Hyperparameters ---
NUM_CLASSES = 300
TOTAL_LANDMARKS = 67  # 15 pose + 2 mid + 42 hands + 8 face
C = 4                 # Channels: x, y, z, confidence
T_FRAMES = 128        # Target number of frames per video
BATCH_SIZE = 32
EPOCHS = 350          # More epochs for a larger dataset
BASE_LR = 3e-4
WEIGHT_DECAY = 1e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
USE_AMP = True        # Automatic Mixed Precision (faster training)
MIXUP_ALPHA = 0.4     # Mixup augmentation strength
LABEL_SMOOTHING = 0.1
NUM_WORKERS = 4       # Adjust based on your system

# --- Reproducibility ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

### --- 3. Preprocessing & Augmentation ---

In [ ]:
def temporal_resample(sequence, target_len):
    """
    Resamples a sequence (T, V, C) to (target_len, V, C)
    using linear interpolation per coordinate.
    """
    orig_len = sequence.shape[0]
    if orig_len == target_len:
        return sequence
    if orig_len == 1:
        return np.repeat(sequence, target_len, axis=0)
    
    # Generate target frame indices
    target_positions = np.linspace(0, orig_len - 1, target_len)
    old_idx = np.arange(orig_len)
    
    # Interpolate each channel (x, y, z, c) for each landmark
    out = np.zeros((target_len, sequence.shape[1], sequence.shape[2]), dtype=sequence.dtype)
    for v in range(sequence.shape[1]): # Iterate landmarks
        for c in range(sequence.shape[2]): # Iterate channels
            values = sequence[:, v, c]
            out[:, v, c] = np.interp(target_positions, old_idx, values)
    return out

class LandmarkPreprocessor:
    """
    Handles normalization and augmentation of the landmark data.
    - Normalization: Makes the data robust to signer's position/size.
    - Augmentation: Creates new variations of data during training.
    """
    def __init__(self, augment=True, target_frames=T_FRAMES):
        self.augment = augment
        self.target_frames = target_frames
        # Landmark indices (must match 'extract_landmarks.ipynb')
        self.POSE_IDXS = list(range(0, 17))
        self.LEFT_HAND_IDXS = list(range(17, 38))
        self.RIGHT_HAND_IDXS = list(range(38, 59))
        self.FACE_IDXS = list(range(59, 67))

    def normalize_frame(self, frame):
        """Normalizes a single frame of landmarks."""
        normalized = frame.copy()
        
        # --- Pose + face normalization (relative to nose/shoulders) ---
        try:
            pose = frame[self.POSE_IDXS, :2]
            if np.sum(np.abs(pose)) > 0:
                nose = pose[0]
                l_s = pose[1]  # Left shoulder
                r_s = pose[2]  # Right shoulder
                
                if np.all(l_s == 0) or np.all(r_s == 0):
                    l_s = np.nan_to_num(np.mean(pose, axis=0))
                    r_s = l_s + 0.1
                dist = np.linalg.norm(l_s - r_s)
            except Exception:
                dist = 0.1
                nose = np.nan_to_num(np.mean(pose, axis=0))
            
            if dist < 1e-4: dist = 0.1
            hu = dist / 2.0
            bw = 6 * hu
            bh = 7 * hu
            bx = nose[0] - 3 * hu
            by = nose[1] - 0.5 * hu
            
            pf_idxs = self.POSE_IDXS + self.FACE_IDXS
            if bw > 0 and bh > 0:
                normalized[pf_idxs, 0] = (frame[pf_idxs, 0] - bx) / bw - 0.5
                normalized[pf_idxs, 1] = (frame[pf_idxs, 1] - by) / bh - 0.5

        # --- Hand normalization (relative to each hand's bounding box) ---
        for h_idxs in [self.LEFT_HAND_IDXS, self.RIGHT_HAND_IDXS]:
            hand = frame[h_idxs]
            if np.sum(np.abs(hand)) > 0:
                xs = hand[:, 0]; ys = hand[:, 1]
                if np.max(xs) != np.min(xs) or np.max(ys) != np.min(ys):
                    xmin, xmax, ymin, ymax = np.min(xs), np.max(xs), np.min(ys), np.max(ys)
                    box_size = max(xmax - xmin, ymax - ymin, 1e-4)
                    normalized[h_idxs, 0] = (hand[:, 0] - xmin) / box_size - 0.5
                    normalized[h_idxs, 1] = (hand[:, 1] - ymin) / box_size - 0.5

        # --- Z-axis normalization (relative to torso) ---
        try:
            torso_z = frame[self.POSE_IDXS, 2]
            torso_z_nonzero = torso_z[torso_z != 0]
            if len(torso_z_nonzero) > 0:
                median_z = np.median(torso_z_nonzero)
                normalized[:, 2] = (frame[:, 2] - median_z)
        except Exception:
            pass
        
        return normalized

    def augment_sequence(self, seq):
        """Applies augmentations to a full sequence."""
        # --- Aug: Rotation noise ---
        angle = random.uniform(-12, 12)
        rad = math.radians(angle)
        cos, sin = math.cos(rad), math.sin(rad)
        R = np.array([[cos, -sin], [sin, cos]], dtype=np.float32)
        xy = seq[:, :, :2].reshape(-1, 2)
        xy = xy @ R.T
        seq[:, :, :2] = xy.reshape(seq.shape[0], seq.shape[1], 2)
        # --- Aug: Add small random noise ---
        seq += np.random.normal(0, 0.003, size=seq.shape).astype(np.float32)

        # --- Aug: Temporal jitter (drop/repeat frames) ---
        if random.random() < 0.5:
            T = seq.shape[0]
            jittered = []
            for t in range(T):
                if random.random() < 0.05 and t < T - 1:
                    continue # 5% chance to drop frame
                jittered.append(seq[t])
                if random.random() < 0.05:
                    jittered.append(seq[t]) # 5% chance to repeat frame
            if len(jittered) < 2:
                jittered = [seq[t] for t in range(T)]
            jittered = np.stack(jittered, axis=0)
            seq = temporal_resample(jittered, self.target_frames) # Resample back
        return seq

    def __call__(self, landmark_sequence: np.ndarray, is_train: bool):
        """Applies the full preprocessing pipeline."""
        # Clean NaNs and resample time
        landmark_sequence = np.nan_to_num(landmark_sequence, 0.0).astype(np.float32)
        seq = temporal_resample(landmark_sequence, self.target_frames)
        
        # Normalize each frame
        seq = np.stack([self.normalize_frame(frame) for frame in seq], axis=0)
        
        # Apply augmentations only if it's training data
        if is_train and self.augment:
            seq = self.augment_sequence(seq)
            
        return seq

### --- 4. PyTorch Dataset ---

In [ ]:
class WLASLLandmarkDataset(Dataset):
    """
    A PyTorch Dataset class to load landmarks and labels.
    
    💡 Note: This class is simpler than the 100-class version.
    The 'nslt_300.json' file already uses contiguous labels from 0-299.
    Therefore, we do *not* need to build a 'label_map'.
    """
    def __init__(self, landmark_path, split_file_path, split='train', preprocessor=None):
        self.split = split
        self.preprocessor = preprocessor if preprocessor is not None else LandmarkPreprocessor(augment=False)
        
        # Load the big landmark file
        landmarks_data = dict(np.load(landmark_path, allow_pickle=True))
        
        # Load the train/val/test split definition
        with open(split_file_path, 'r') as f:
            split_data = json.load(f)
        split_dict = split_data.get('root', split_data)
        
        self.samples = []
        for video_id, info in split_dict.items():
            # Check if this video is in the correct split (train, val, or test)
            subset = info.get('subset') or info.get('split') or info.get('subset', None)
            if subset == self.split:
                # Check if we have landmarks for this video
                if video_id in landmarks_data:
                    action = info.get('action', [])
                    if not isinstance(action, list) or len(action) == 0:
                        continue
                    
                    # The label is just the integer value of 'action[0]'
                    try:
                        label = int(action[0])
                    except Exception:
                        continue # Skip if label is not an integer
                        
                    landmarks = landmarks_data[video_id]
                    self.samples.append({"video_id": video_id, "landmarks": landmarks, "label": label})
                    
        print(f"✅ Created '{self.split}' split with {len(self.samples)} samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        landmark_sequence = sample["landmarks"]
        
        # Handle a legacy flattened format
        if landmark_sequence.ndim == 2 and landmark_sequence.shape[1] == TOTAL_LANDMARKS * 3:
            landmark_sequence = landmark_sequence.reshape(-1, TOTAL_LANDMARKS, 3)
            
        # Apply normalization and augmentation
        processed = self.preprocessor(landmark_sequence, is_train=(self.split == 'train'))
        
        # Transpose to (C, T, V) format for PyTorch Conv2D layers
        processed = np.transpose(processed, (2, 0, 1)).astype(np.float32)
        
        label = int(sample['label'])
        
        return torch.from_numpy(processed), torch.tensor(label, dtype=torch.long)

def collate_fn(batch):
    """Custom collate function to stack batch data."""
    xs, ys = zip(*batch)
    xs = torch.stack(xs, dim=0)
    ys = torch.tensor(ys, dtype=torch.long)
    return xs, ys

### --- 5. ST-GCN Model ---

In [ ]:
class GraphConv(nn.Module):
    """
    Spatial Graph Convolution layer.
    """
    def __init__(self, in_channels, out_channels, num_nodes, bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.theta = nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.A = nn.Parameter(torch.eye(num_nodes, dtype=torch.float32), requires_grad=True)
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_channels))
        else:
            self.register_parameter('bias', None)
        nn.init.xavier_uniform_(self.theta)

    def forward(self, x):
        B, C, T, V = x.shape
        x_perm = x.permute(0, 2, 3, 1).contiguous().view(B * T, V, C)
        y = x_perm @ self.theta
        A = self.A.unsqueeze(0)
        y = torch.bmm(A.repeat(B * T, 1, 1), y)
        y = y.view(B, T, V, self.out_channels).permute(0, 3, 1, 2).contiguous()
        if self.bias is not None:
            y = y + self.bias.view(1, -1, 1, 1)
        return y

class STGCNBlock(nn.Module):
    """
    Spatio-Temporal Graph Convolutional (ST-GCN) block.
    """
    def __init__(self, in_channels, out_channels, num_nodes, kernel_size=9, stride=1, residual=True):
        super().__init__()
        self.gcn = GraphConv(in_channels, out_channels, num_nodes)
        padding = (kernel_size - 1) // 2
        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=(kernel_size, 1), padding=(padding, 0), stride=(stride, 1)),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(0.3)
        )
        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and stride == 1:
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride,1)),
                nn.BatchNorm2d(out_channels),
            )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        res = self.residual(x)
        x = self.gcn(x)
        x = self.tcn(x)
        x = x + res
        return self.relu(x)

class TemporalSelfAttention(nn.Module):
    """
    Applies self-attention across the TIME dimension.
    """
    def __init__(self, d_model, n_heads=4):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, n_heads, batch_first=True)
        self.ln = nn.LayerNorm(d_model)

    def forward(self, x):
        B, C, T, V = x.shape
        x = x.permute(0, 3, 2, 1).contiguous().view(B * V, T, C)
        out, _ = self.attn(x, x, x)
        out = self.ln(out + x)
        out = out.view(B, V, T, C).permute(0, 3, 2, 1).contiguous()
        return out

class STGCN(nn.Module):
    """
    The full ST-GCN model with Temporal Self-Attention.
    """
    def __init__(self, in_channels, num_class, num_nodes):
        super().__init__()
        self.data_bn = nn.BatchNorm1d(in_channels * num_nodes)
        self.layers = nn.ModuleList([
            STGCNBlock(in_channels, 64, num_nodes, kernel_size=9, stride=1, residual=False),
            STGCNBlock(64, 64, num_nodes, kernel_size=9, stride=1),
            STGCNBlock(64, 128, num_nodes, kernel_size=9, stride=2),
            STGCNBlock(128, 256, num_nodes, kernel_size=9, stride=2),
        ])
        self.temporal_attn = TemporalSelfAttention(d_model=256, n_heads=4)
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(256, num_class)

    def forward(self, x):
        B, C, T, V = x.shape
        x = x.permute(0, 1, 3, 2).contiguous()
        x = x.view(B, C * V, T)
        x = self.data_bn(x)
        x = x.view(B, C, V, T).permute(0, 1, 3, 2).contiguous()
        for layer in self.layers:
            x = layer(x)
        x = self.temporal_attn(x)
        x = self.pool(x)
        x = x.view(B, -1)
        out = self.fc(x)
        return out

# --------------------------
# --- 6. Metrics & Mixup ---
# --------------------------

def mixup_data(x, y, alpha=1.0, device=None):
    """
    Applies Mixup augmentation.
    """
    if alpha <= 0:
        return x, y, y, 1.0
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def accuracy_topk(output, target, topk=(1,5)):
    """
    Computes the accuracy over the k top predictions.
    💡 This is the improved function that calculates Top-1 and Top-5.
    """
    maxk = max(topk)
    batch_size = target.size(0)
    
    _, pred = output.topk(maxk, 1, True, True) # Get top-k indices
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred)) # Compare
    
    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append((correct_k.mul_(100.0 / batch_size)).item())
    return res  # [top1_acc (%), top5_acc (%)]

def count_parameters(model):
    """Counts the number of trainable parameters in a model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### --- 7. Training Pipeline ---

In [ ]:
def main():
    # --- 1. Create Output Directory ---
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"All outputs will be saved to: {OUTPUT_DIR}")

    # --- 2. Create Datasets & Loaders ---
    print("Initializing preprocessors...")
    preprocessor = LandmarkPreprocessor(augment=True, target_frames=T_FRAMES)
    val_preprocessor = LandmarkPreprocessor(augment=False, target_frames=T_FRAMES) # No augmentation for validation

    print("Loading datasets...")
    train_dataset = WLASLLandmarkDataset(LANDMARK_FILE, SPLIT_FILE, split='train', preprocessor=preprocessor)
    val_dataset = WLASLLandmarkDataset(LANDMARK_FILE, SPLIT_FILE, split='val', preprocessor=val_preprocessor)
    test_dataset = WLASLLandmarkDataset(LANDMARK_FILE, SPLIT_FILE, split='test', preprocessor=val_preprocessor)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, collate_fn=collate_fn)

    # --- 3. Compute Class Weights ---
    print("Calculating class weights...")
    train_labels = [int(s['label']) for s in train_dataset.samples]
    if len(train_labels) == 0:
        raise RuntimeError('No training samples found. Check your split file and landmark file paths.')
    # Assumes labels are 0 to NUM_CLASSES-1
    class_weights = compute_class_weight('balanced', classes=np.arange(NUM_CLASSES), y=train_labels)
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

    # --- 4. Initialize Model, Optimizer, Loss ---
    print("Initializing model...")
    model = STGCN(in_channels=C, num_class=NUM_CLASSES, num_nodes=TOTAL_LANDMARKS).to(DEVICE)
    print(f"Model has {count_parameters(model)} trainable parameters.")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
    
    # 💡 Bug Fix: Using CosineAnnealingLR, so scheduler.step() needs no arguments.
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=LABEL_SMOOTHING)

    # --- 5. Start Training Loop ---
    best_val = 0.0
    # 💡 Added 'val_top5' to history
    train_history = {'epoch': [], 'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_top5': []}
    
    print(f"--- Starting training for {EPOCHS} epochs on {DEVICE} ---")
    for epoch in range(EPOCHS):
        # --- Training Phase ---
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        total_samples = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
        for x, y in pbar:
            x = x.to(DEVICE); y = y.to(DEVICE)
            batch_size = x.size(0)
            
            if MIXUP_ALPHA > 0:
                mixed_x, y_a, y_b, lam = mixup_data(x, y, alpha=MIXUP_ALPHA, device=DEVICE)
            else:
                mixed_x, y_a, y_b, lam = x, y, y, 1.0

            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=USE_AMP):
                logits = model(mixed_x)
                loss = lam * criterion(logits, y_a) + (1 - lam) * criterion(logits, y_b)
                
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            # 💡 Use new accuracy function
            acc1 = accuracy_topk(logits.detach().cpu(), y.detach().cpu(), topk=(1,))[0]
            running_loss += loss.item() * batch_size
            running_acc += acc1 * batch_size / 100.0
            total_samples += batch_size
            pbar.set_postfix(loss=running_loss / total_samples, acc=(running_acc / total_samples * 100.0))

        avg_train_loss = running_loss / len(train_dataset)
        avg_train_acc = running_acc / len(train_dataset) * 100.0

        # --- Validation Phase ---
        model.eval()
        val_loss = 0.0
        val_acc = 0.0
        val_top5 = 0.0 # 💡 Added Top-5
        with torch.no_grad():
            for x, y in tqdm(val_loader, desc="Validation"):
                x = x.to(DEVICE); y = y.to(DEVICE)
                with torch.cuda.amp.autocast(enabled=USE_AMP):
                    logits = model(x)
                    loss = criterion(logits, y)
                    
                val_loss += loss.item() * x.size(0)
                # 💡 Use new accuracy function for Top-1 and Top-5
                acc1, acc5 = accuracy_topk(logits.detach().cpu(), y.detach().cpu(), topk=(1,5))
                val_acc += acc1 * x.size(0) / 100.0
                val_top5 += acc5 * x.size(0) / 100.0
                
        avg_val_loss = val_loss / len(val_dataset) if len(val_dataset) > 0 else 0.0
        avg_val_acc = val_acc / len(val_dataset) * 100.0 if len(val_dataset) > 0 else 0.0
        avg_val_top5 = val_top5 / len(val_dataset) * 100.0 if len(val_dataset) > 0 else 0.0

        # 💡 Bug Fix: Call scheduler.step() with no arguments
        scheduler.step()

        # 💡 Print Top-5 accuracy
        print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss {avg_train_loss:.4f} Acc {avg_train_acc:.2f}% | Val Loss {avg_val_loss:.4f} Acc {avg_val_acc:.2f}% Top5 {avg_val_top5:.2f}%")

        # Log history
        train_history['epoch'].append(epoch+1)
        train_history['train_loss'].append(avg_train_loss)
        train_history['train_acc'].append(avg_train_acc)
        train_history['val_loss'].append(avg_val_loss)
        train_history['val_acc'].append(avg_val_acc)
        train_history['val_top5'].append(avg_val_top5) # 💡 Log Top-5

        # Save the best model
        if avg_val_acc > best_val:
            best_val = avg_val_acc
            save_path = os.path.join(OUTPUT_DIR, f"best_stgcn_{NUM_CLASSES}.pth")
            torch.save(model.state_dict(), save_path)
            print(f"✅ New best model saved: {best_val:.2f}%")

    print(f"Training complete. Best val acc: {best_val:.2f}%")

    # --- 6. Save History & Plots ---
    df = pd.DataFrame(train_history)
    df.to_csv(os.path.join(OUTPUT_DIR, 'training_log.csv'), index=False)
    print(f"Training log saved to {os.path.join(OUTPUT_DIR, 'training_log.csv')}")

    # Plot curves
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(df['epoch'], df['train_loss'], label='Train Loss')
    plt.plot(df['epoch'], df['val_loss'], label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.title('Loss Curve')
    plt.subplot(1,2,2)
    plt.plot(df['epoch'], df['train_a'c'], label='Train Acc')
    plt.plot(df['epoch'], df['val_acc'], label='Val Acc')
    plt.plot(df['epoch'], df['val_top5'], label='Val Top-5 Acc', linestyle='--') # 💡 Plot Top-5
    plt.xlabel('Epoch'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.title('Accuracy Curve')
    plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves.png'))
    plt.close()
    print(f"Training curves saved to {os.path.join(OUTPUT_DIR, 'training_curves.png')}")

    # --- 7. Final Test ---
    print("Loading best model for final testing...")
    save_path = os.path.join(OUTPUT_DIR, f"best_stgcn_{NUM_CLASSES}.pth")
    model.load_state_dict(torch.load(save_path, map_location=DEVICE))
    model.eval()
    test_acc = 0.0
    test_top5 = 0.0 # 💡 Added Top-5
    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="Testing"):
            x = x.to(DEVICE); y = y.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=USE_AMP):
                logits = model(x)
            # 💡 Use new accuracy function
            acc1, acc5 = accuracy_topk(logits.detach().cpu(), y.detach().cpu(), topk=(1,5))
            test_acc += acc1 * x.size(0) / 100.0
            test_top5 += acc5 * x.size(0) / 100.0
            
    avg_test_acc = test_acc / len(test_dataset) * 100.0 if len(test_dataset) > 0 else 0.0
    avg_test_top5 = test_top5 / len(test_dataset) * 100.0 if len(test_dataset) > 0 else 0.0
    
    print(f"--- 🏁 Final Test Accuracy ---")
    print(f"Top-1: {avg_test_acc:.2f}%")
    print(f"Top-5: {avg_test_top5:.2f}%") 

In [ ]:

if __name__ == "__main__":
    main()

✅ Created 'train' split with 3549 samples.
✅ Created 'val' split with 901 samples.


/tmp/ipykernel_36/1706839668.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)


✅ Created 'test' split with 668 samples.
Model has 1260776 trainable parameters.


Epoch 1/350 Train:   0%|          | 0/111 [00:00<?, ?it/s]/tmp/ipykernel_36/1706839668.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=USE_AMP):
Validation:   0%|          | 0/29 [00:00<?, ?it/s]/tmp/ipykernel_36/1706839668.py:65: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=USE_AMP):
Validation: 100%|██████████| 29/29 [00:12<00:00,  2.38it/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to re

Epoch 1/350 | Train Loss 5.7042 Acc 0.34% | Val Loss 5.4844 Acc 1.44%
✅ New best model: 1.44%


...

Validation: 100%|██████████| 29/29 [00:12<00:00,  2.39it/s]


Epoch 296/350 | Train Loss 1.8745 Acc 57.51% | Val Loss 2.1642 Acc 70.03%


Validation: 100%|██████████| 29/29 [00:12<00:00,  2.41it/s]


Epoch 297/350 | Train Loss 1.7310 Acc 48.41% | Val Loss 2.1407 Acc 71.14%


Epoch 298/350 Train:   7%|▋         | 8/111 [00:04<00:35,  2.86it/s, acc=19.1, loss=1.42]